## Numpy를 사용하여 RNN 동작 이해

In [95]:
import numpy as np

# RNN이 몇 번 펼쳐질 것인가? => 문장의 길이
timesteps = 10  # 문장의 길이 t

# RNN의 입력. 일반적으로는 단어의 벡터 차원
input_size = 4  # 단어의 차원 D

# RNN cell에서 hidden unit의 갯수 (cell 용량)
hidden_size = 8

In [96]:
# RNN 입력 데이터 (projection layer)
inputs = np.random.random((timesteps, input_size))  # t x D

# hidden state
hidden_state_t = np.zeros((hidden_size,))  # h_0 = 0

In [97]:
inputs

array([[0.55005779, 0.70699418, 0.27868454, 0.40742211],
       [0.40851319, 0.92599005, 0.09660356, 0.67034797],
       [0.00240994, 0.45084885, 0.38246113, 0.17397006],
       [0.8817862 , 0.67123798, 0.54660679, 0.07816789],
       [0.56690557, 0.95537761, 0.93295766, 0.30609235],
       [0.33600518, 0.1628346 , 0.00200744, 0.96494052],
       [0.92713887, 0.77960703, 0.2159499 , 0.22218719],
       [0.11635071, 0.51666084, 0.91775734, 0.64956564],
       [0.39259072, 0.65503127, 0.87092849, 0.52694392],
       [0.38869996, 0.08502068, 0.7066489 , 0.7412308 ]])

In [98]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [99]:
# RNN cell 뉴런의 가중치 설정
# 1. 입력 x_t 와 대응되는 가중치 (D x hidden_size)
# 2. 이전 시점의 상태인 h_(t-1) 에 대응되는 가중치 (hidden_size x hidden_size)

W_x = np.random.random((hidden_size, input_size))
W_h = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size,))

$$
h_t = tanh(W_xx_t + W_hh_{t-1} + b)
$$

In [100]:
total_hidden_states = []

# RNN 작동

# 단어 벡터 하나씩 순서대로 꺼낸다
for input_t in inputs:
    # output_t 가 실제로는 h_t 의 역할 (현 시점의 hidden state)
    output_t = np.tanh(W_x @ input_t + W_h @ hidden_state_t + b)

    # 각 시점의 은닉 상태의 값을 계속해서 기록
    total_hidden_states.append(list(output_t))

    hidden_state_t = output_t

# 출력 시 값을 깔끔하게 만들어줌
total_hidden_states = np.stack(total_hidden_states, axis=0)

print(total_hidden_states)

[[0.95970007 0.89367712 0.91889597 0.90383027 0.8924485  0.92634026
  0.94813164 0.86547663]
 [0.99997725 0.99997739 0.99994938 0.999922   0.99947753 0.99992674
  0.99996446 0.99997044]
 [0.99992801 0.99997755 0.99994725 0.99965346 0.99945178 0.9997455
  0.99991737 0.99997384]
 [0.99998823 0.99999399 0.99996711 0.99994117 0.99976263 0.99985392
  0.99993988 0.9999846 ]
 [0.99998902 0.99999675 0.99998992 0.99996147 0.99987686 0.99993557
  0.99997241 0.99999139]
 [0.99997931 0.99998724 0.99993936 0.99991908 0.99933586 0.99992355
  0.99996794 0.99997649]
 [0.9999915  0.99999183 0.99995994 0.99995904 0.99969848 0.99990051
  0.99995682 0.99998384]
 [0.99997306 0.99999548 0.99998664 0.99991275 0.99979971 0.99992444
  0.99997009 0.99998842]
 [0.99998382 0.99999617 0.99998667 0.99994432 0.99982861 0.99992945
  0.99997055 0.9999894 ]
 [0.99997623 0.99999522 0.9999704  0.99990742 0.99968887 0.99988985
  0.99995517 0.99998321]]


## PyTorch 를 이용한 RNN

In [101]:
import torch
import torch.nn as nn

In [102]:
input_size = 5  # 입력되는 단어 벡터의 차원
hidden_size = 8  # 셀의 용량

`[I, am, a , Student]`라는 문장이 들어간다고 가정


In [103]:
# (batch_size, timesteps, input_size) ==> (배치 크기, 문장 길이, 단어 벡터 크기)
inputs = torch.Tensor(1, 4, input_size)
print(inputs)

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


In [104]:
# (데이터의 개수/ 배치 크기, 문장의 길이, 단어 벡터의 차원)
inputs.shape

torch.Size([1, 4, 5])

In [105]:
# RNN 레이어에 필요한 것 => 입력 데이터 차원, 뉴런 갯수
cell = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

In [106]:
outputs, hidden = cell(inputs)

In [107]:
# 모든 timestep 각각의 hidden state
outputs

tensor([[[-0.5376,  0.0150,  0.3870,  0.1376, -0.2885,  0.2072,  0.0701,
          -0.0450],
         [-0.3234,  0.1055,  0.4030, -0.1511, -0.2127,  0.0938, -0.0725,
           0.1532],
         [-0.4150,  0.0925,  0.2726, -0.0114, -0.2602, -0.0357, -0.0664,
           0.0636],
         [-0.4399,  0.0600,  0.3075, -0.0010, -0.2849,  0.0260, -0.0283,
           0.0419]]], grad_fn=<TransposeBackward1>)

In [108]:
# 제일 마지막 timestep의 hidden state
hidden

tensor([[[-0.4399,  0.0600,  0.3075, -0.0010, -0.2849,  0.0260, -0.0283,
           0.0419]]], grad_fn=<StackBackward0>)

## many to one, many to many

- Many to One은 여러 timestep을 입력 받아 하나의 결과를 낸다.
- Many to Many는 여러 timestep을 입력 받아 여러 결과를 낸다.

## Deep RNN (Deep Recurrent Neural Network)
RNN 층이 여러 겹으로 쌓여있는 경우

In [109]:
# ( batch_size, timestep, input_size )
inputs = torch.Tensor(1, 4, 5)

In [110]:
cell = nn.RNN(
    input_size=5, hidden_size=8, batch_first=True, num_layers=2  # RNN 층이 몇개인지
)

outputs, hidden = cell(inputs)

In [111]:
# outputs는 모든 timestep 각각의 hidden state 이므로 층이 몇 층이던 동일
outputs.shape

torch.Size([1, 4, 8])

In [112]:
# 각 층의 마지막 timestep (층 수, batch size, hidden state)
hidden.shape

torch.Size([2, 1, 8])

## Bi-RNN (Bidirectional RNN)

In [113]:
inputs = torch.Tensor(1, 4, 5)

In [114]:
cell = nn.RNN(input_size=5, hidden_size=8, batch_first=True, bidirectional=True)
outputs, hidden = cell(inputs)

In [115]:
# (1, 4, 16) 인 이유 : 순방향, 역방향 hidden state 각 8개
outputs.shape

torch.Size([1, 4, 16])

In [116]:
# hidden[0] : 순방향 hidden state
# hidden[1] : 1 번쨰가 역방향 hidden state

hidden.shape

torch.Size([2, 1, 8])